# function_call模式

In [3]:
import dotenv
import os
from langchain_classic.agents import AgentExecutor
from langchain_classic.agents import create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

# 设置环境变量 Tavily
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 初始化模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Tavily 搜索工具
search = TavilySearchResults(max_results=3)

# 定义 Prompt（新版必须明确 prompt）
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个可以访问互联网信息的智能助手。"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# 创建新版 Agent（替代 initialize_agent）
agent = create_openai_functions_agent(
    llm=llm,
    tools=[search],
    prompt=prompt,
)

# 创建执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search],
    verbose=True,
)

# 运行
result = agent_executor.invoke({"input": "查询北京今天的天气"})
print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '北京今天的天气'}`


[{'title': '北京发布大雾黄色预警信号今夜至明晨大部能见度小于1000米 - 资讯', 'url': 'http://news.weather.com.cn/2025/10/4383537.shtml', 'content': '2025-10-29 15:48:04 来源： 中国天气网\n\n字号\n\n 大\n 中\n 小\n\n分享到\n\n原创未经授权不得转载\n\n中国天气网讯 北京市气象台2025年10月29日16时00分发布大雾黄色预警信号：预计今天夜间至30日早晨，北京大部分地区能见度小于1000米，东部和南部地区不足500米，请注意防范。\n\n编辑：天气融媒体\n\n分享到\n\n查天气 关注 “中国天气网” 微信公众号、“中国天气”微博、“中国天气”抖音\n\n最新推荐\n\n查看更多\n\n高清图集\n\n 丰收色彩图鉴 世界粮食日看大地美学\n 甘肃定西雾气蒙蒙 城市如盖“云被”\n 爬山虎染秋色 浓烈似火\n\n原创策划\n\n天气视频\n\n 数说气象\n 今后三天精细化降水预报\n 这个冬天会是冷冬吗\n\n生活旅游\n\n 雨后峨眉沟壑尽显 金顶显真容\n 秋意浓 蓝天映衬下的哈尔滨伏尔加庄园\n 大美新疆—帕米尔高原好风光\n\n### 网站服务\n:   关于我们联系我们用户反馈\n\n    版权声明网站律师\n\n### 营销中心\n:   商务合作广告服务媒资合作\n\n### 相关链接\n\n中国气象局中国气象服务协会 中国天气频道', 'score': 0.84707206}, {'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/', 'content': '|  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 | 08:00 |\n|

# ReAct模式
## 方式1：自定义提示词（将仓库中的提示词复制粘贴到代码中）
提示词仓库地址：https://smith.langchain.com/hub/hwchase17

In [7]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_classic.agents import create_react_agent
from langchain_classic.agents import AgentExecutor
from langchain_core.prompts import PromptTemplate
import dotenv, os

dotenv.load_dotenv()

# 设置环境变量 Tavily
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 初始化模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Tavily 搜索工具
search = TavilySearchResults(max_results=3)

# 相较于传统方式，通用方式下的react模式需要自定义提示词
# 远程提示词仓库中有预设好的关于react模式的提示词 https://smith.langchain.com/hub/hwchase17
template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""
prompt = PromptTemplate.from_template(
    template=template
)

# 创建新版 Agent（替代 initialize_agent）
agent = create_react_agent(
    llm=llm,
    tools=[search],
    prompt=prompt,
)

# 创建执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search],
    verbose=True,
    handle_parsing_errors=True
)

# 运行
result = agent_executor.invoke({"input": "查询北京今天的天气"})
print(result["output"])




> Entering new AgentExecutor chain...
Action: tavily_search_results_json  
Action Input: 北京今天的天气  
[{'title': '北京发布大雾黄色预警信号今夜至明晨大部能见度小于1000米 - 资讯', 'url': 'http://news.weather.com.cn/2025/10/4383537.shtml', 'content': '2025-10-29 15:48:04 来源： 中国天气网\n\n字号\n\n 大\n 中\n 小\n\n分享到\n\n原创未经授权不得转载\n\n中国天气网讯 北京市气象台2025年10月29日16时00分发布大雾黄色预警信号：预计今天夜间至30日早晨，北京大部分地区能见度小于1000米，东部和南部地区不足500米，请注意防范。\n\n编辑：天气融媒体\n\n分享到\n\n查天气 关注 “中国天气网” 微信公众号、“中国天气”微博、“中国天气”抖音\n\n最新推荐\n\n查看更多\n\n高清图集\n\n 丰收色彩图鉴 世界粮食日看大地美学\n 甘肃定西雾气蒙蒙 城市如盖“云被”\n 爬山虎染秋色 浓烈似火\n\n原创策划\n\n天气视频\n\n 数说气象\n 今后三天精细化降水预报\n 这个冬天会是冷冬吗\n\n生活旅游\n\n 雨后峨眉沟壑尽显 金顶显真容\n 秋意浓 蓝天映衬下的哈尔滨伏尔加庄园\n 大美新疆—帕米尔高原好风光\n\n### 网站服务\n:   关于我们联系我们用户反馈\n\n    版权声明网站律师\n\n### 营销中心\n:   商务合作广告服务媒资合作\n\n### 相关链接\n\n中国气象局中国气象服务协会 中国天气频道', 'score': 0.84707206}, {'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/', 'content': '|  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 | 08:00 |\n| 天气 |  | 

## 方式2：使用Python SDK拉取提示词

```python
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=LANGSMITH_API_KEY)
prompt = client.pull_prompt("hwchase17/react", include_model=True)
```

In [8]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_classic.agents import create_react_agent
from langchain_classic.agents import AgentExecutor

import dotenv, os

dotenv.load_dotenv()

# 设置环境变量 Tavily
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

# 初始化模型
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Tavily 搜索工具
search = TavilySearchResults(max_results=3)

# 拉取提示词
from langsmith import Client
client = Client(api_key=os.getenv("LANGSMITH_API_KEY"))
prompt = client.pull_prompt("hwchase17/react", include_model=True)

# 创建新版 Agent（替代 initialize_agent）
agent = create_react_agent(
    llm=llm,
    tools=[search],
    prompt=prompt,
)

# 创建执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search],
    verbose=True,
    handle_parsing_errors=True
)

# 运行
result = agent_executor.invoke({"input": "查询北京今天的天气"})
print(result["output"])




> Entering new AgentExecutor chain...
我需要查找北京今天的天气情况。  
Action: tavily_search_results_json  
Action Input: "北京今天的天气"  [{'title': '北京发布大雾黄色预警信号今夜至明晨大部能见度小于1000米 - 资讯', 'url': 'http://news.weather.com.cn/2025/10/4383537.shtml', 'content': '2025-10-29 15:48:04 来源： 中国天气网\n\n字号\n\n 大\n 中\n 小\n\n分享到\n\n原创未经授权不得转载\n\n中国天气网讯 北京市气象台2025年10月29日16时00分发布大雾黄色预警信号：预计今天夜间至30日早晨，北京大部分地区能见度小于1000米，东部和南部地区不足500米，请注意防范。\n\n编辑：天气融媒体\n\n分享到\n\n查天气 关注 “中国天气网” 微信公众号、“中国天气”微博、“中国天气”抖音\n\n最新推荐\n\n查看更多\n\n高清图集\n\n 丰收色彩图鉴 世界粮食日看大地美学\n 甘肃定西雾气蒙蒙 城市如盖“云被”\n 爬山虎染秋色 浓烈似火\n\n原创策划\n\n天气视频\n\n 数说气象\n 今后三天精细化降水预报\n 这个冬天会是冷冬吗\n\n生活旅游\n\n 雨后峨眉沟壑尽显 金顶显真容\n 秋意浓 蓝天映衬下的哈尔滨伏尔加庄园\n 大美新疆—帕米尔高原好风光\n\n### 网站服务\n:   关于我们联系我们用户反馈\n\n    版权声明网站律师\n\n### 营销中心\n:   商务合作广告服务媒资合作\n\n### 相关链接\n\n中国气象局中国气象服务协会 中国天气频道', 'score': 0.84707206}, {'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/', 'content': '|  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 | 